---
title: Representing dynamic models
format:
  html: 
    toc: true
    warning: false
    error: false
code-line-numbers: true
---

*simulib* provides a structure capable of representing dynamic metabolic models. The overarching data structure encoding an entire model is the `DynamicSimulationInput` dataclass.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "/app/src")

### Dynamic fluxes

One of the main components of `DynamicSimulationInput` is the `ODE` class representing a single variable associated with an expression that describes its consumption or production over time.

`ODE` instances can be easily created from a variety of inputs. At its core, `ODE` must contain a variable, initial condition, and a rhs expression.
`annotations` is an optional dict that users can use to populate information about the ODE.
`ODESimulationProperties` is an optional field that users can use add the `abs_tolerance`.

In [2]:
from simulib.entities.dynamic import ODE, ODESimulationProperties

glucose_ode = ODE(
    variable="Glucose",
    metabolite_id=None,
    initial_condition=10,
    annotations={},
    simulation_properties=ODESimulationProperties(abs_tolerance=1e-3),
    rhs_expression=[
        {
            "expr": "Biomass * (1/Glucose)", 
            "cond": "Glucose < 10"
        },
        {
            "expr": 
                [
                    {
                        "expr": "Biomass * (2/Glucose)",
                        "cond": "Pulse = 1"
                    },
                    {
                        "expr": "Biomass * (3/Glucose)",
                        "cond": "Pulse = 2"                    
                    }
                ],
            "cond": "Glucose > 10"
        }
    ]
)
display(glucose_ode)

ODE(variable='Glucose', rhs_expression=Piecewise((Biomass/Glucose, Glucose < 10), (Piecewise((2*Biomass/Glucose, Eq(Pulse, 1)), (3*Biomass/Glucose, Eq(Pulse, 2))), Glucose > 10)), initial_condition=10.0000000000000, annotations={}, simulation_properties=ODESimulationProperties(abs_tolerance=0.001), metabolite_id=None)

Optionally, the metabolite described by a `ODE` instance can be associated with an exchange flux in another model. This relationship can be captured through the `DynamicExchangeFlux` class. These fluxes can have lower and upper bounds defined by expressions.

Note that on `DynamicExchangeFlux` instances, upper and lower bound expressions must have a non-null `cond` field.

In [3]:
from simulib.entities.dynamic import DynamicExchangeFlux, DynamicExpression
glucose_variable = glucose_ode.variable

glucose_exchange = DynamicExchangeFlux(
    exchange_flux_id = "EX_glc__D_e",
    upper_bound = 
        DynamicExpression.from_object(
            {"expr": f"{glucose_variable} * 2", "cond": f"{glucose_variable}"}
        )
)
display(glucose_exchange)

DynamicExchangeFlux(exchange_flux_id='EX_glc__D_e', lower_bound=None, upper_bound=DynamicExpression(expr=2*Glucose, cond=Glucose))

## Dynamic simulation inputs

`DynamicSimulationInput` objects are used to represent a dynamic model. The components required for this are:

- name: str: The name of the model.
- description: Optional[str]: A description of the model.
- odes: List[ODE]: A list of ODEs in the model.
- exchange_fluxes: List[DynamicExchangeFlux]: A list of exchange fluxes.
- variables: Dict[str, ValueOrExpr]: A dictionary of variables and their definitions. These are variables that are not ODEs but are used in the expressions.
- simulation_properties: Dict[str, Union[str, int, float, bool]]: General simulation properties.

It has the following Key Methods:
- from_dict: Creates a DynamicModelInput from a dictionary.
- validate_expression_dicts: Pydantic validator to parse the variables dictionary.
- initial_conditions: Returns a dictionary of initial conditions for the ODEs.
- free_variables: Returns the free variables in the model.
- exchange_variables: Returns the exchange reaction IDs.
- defined_variables: Returns the names of the defined variables.

Each component is parsed using the function defined in its class (e.g. `from_dict`, `from_object`). This means that entire models can be stored as JSON or YAML inputs, converted into dictionaries of strings and parsed into a `DynamicSimulationInput`. In the following example, we combine all of the features explored in the previous section to easily import a model

In [4]:
from yaml import safe_load as yaml_load

with open("/app/resources/models/toy_scere_glc_growth.yaml", "r") as f:
    dynamic_model_dict = yaml_load(f)

dynamic_model_dict

{'name': 'Toy S. cerevisae Glucose Growth Model',
 'description': 'Simulib-ready toy model',
 'simulation_properties': {'algorithm': 'direct',
  'display': 'none',
  'ode_method': 'BDF',
  'rel_tolerance': '1e-4',
  'tout': 0.001,
  'tstart': 0,
  'tstop': 24},
 'variables': {'Vgmax': 8.5,
  'Kg': 0.5,
  'D': 0.044,
  'Gin': 100.0,
  'Vomax': 8.0,
  'Ko': 0.01},
 'odes': [{'initial_condition': 0.5,
   'metabolite_id': None,
   'rhs_expression': 'D',
   'variable': 'Volume'},
  {'initial_condition': 0.05,
   'metabolite_id': None,
   'rhs_expression': 'BIOMASS_SC4_bal * Biomass - D * Biomass / Volume',
   'variable': 'Biomass'},
  {'initial_condition': 0.05,
   'metabolite_id': None,
   'rhs_expression': 'BIOMASS_SC4_bal * Biomass - D * Biomass / Volume',
   'variable': 'Biomass'},
  {'initial_condition': 10.0,
   'metabolite_id': None,
   'rhs_expression': 'EX_glc__D_e * Biomass + D * (Gin - Glucose) / Volume',
   'variable': 'Glucose'},
  {'initial_condition': 0.0,
   'metabolite_id':

In [5]:
from simulib.entities.dynamic import DynamicModelInput

ds_input = DynamicModelInput.from_dict(dynamic_model_dict)
ds_input

DynamicModelInput(name='Toy S. cerevisae Glucose Growth Model', description='Simulib-ready toy model', odes=[ODE(variable='Volume', rhs_expression=D, initial_condition=0.500000000000000, annotations={}, simulation_properties=ODESimulationProperties(abs_tolerance=None), metabolite_id=None), ODE(variable='Biomass', rhs_expression=BIOMASS_SC4_bal*Biomass - Biomass*D/Volume, initial_condition=0.0500000000000000, annotations={}, simulation_properties=ODESimulationProperties(abs_tolerance=None), metabolite_id=None), ODE(variable='Biomass', rhs_expression=BIOMASS_SC4_bal*Biomass - Biomass*D/Volume, initial_condition=0.0500000000000000, annotations={}, simulation_properties=ODESimulationProperties(abs_tolerance=None), metabolite_id=None), ODE(variable='Glucose', rhs_expression=Biomass*EX_glc__D_e + D*(Gin - Glucose)/Volume, initial_condition=10.0000000000000, annotations={}, simulation_properties=ODESimulationProperties(abs_tolerance=None), metabolite_id=None), ODE(variable='Ethanol', rhs_expr